## PetFinder

In [ ]:
import os
import tqdm
import torch

import numpy as np
import pandas as pd

Проверяем работает ли GPU

In [ ]:
!nvidia-smi

In [ ]:
!ping -c 4 google.com

## Предобработка данных или разведочный анализ (EDA)

In [ ]:
!ls ../input # Посмотрим что внутри директории input

In [ ]:
!ls ../input/petfinder-adoption-prediction/train # Заглянем в папку train

In [ ]:
train = pd.read_csv('../input/petfinder-adoption-prediction/train/train.csv') # Прочитаем файл
train.head(4)

In [ ]:
train.shape # Посмотрим на размеры табличных данных (23 фичи, 24-ая это искомая переменная)

In [ ]:
train.info() # Просмотрим типы данных

Тип данных dtype (число с плавающей точкой) в колонке PhotoAmt (количество фото) выглядит странным! Изменим тип на np.int64 так, чтобы оно соотвествовало остальным колонкам

In [ ]:
train.PhotoAmt = train.PhotoAmt.astype(np.int64)

Посмотрим на распределение целевой колонки (количества строк по каждому классу):

In [ ]:
np.unique(train.AdoptionSpeed, return_counts=True)

Промежуточные выводы/предположения:
1. Сразу мало кто разбирает животных
2. Основными фичами, коррелирующими с целевой переменной скорее всего будут: здоровье, возраст (относительно для данной породы), привито/не привито, фото животного 

Удаляем текстовые данные (фичи), которые не оказывают влияния на распределение целевых значений

In [ ]:
def filter_text_columns(table):
    _blacklist = ['Name', 'RescuerID', 'Description', 'PetID']
    for column in _blacklist:
        if column in table.columns:
            del table[column]

filter_text_columns(train)

Разделим датасеты тренировочных данных и целевых данных (искомые результаты)

In [ ]:
X = np.array(train.iloc[:,:-1])  # Берем все строки кроме последней (целевой колонки)
y = np.array(train.AdoptionSpeed)

In [ ]:
assert X.shape == (14993, 19)  # Проверяем, чтобы форма Х сета была той которую мы ожидаем, без потери каких-либо данных
assert y.shape == (14993,)  # Проверка целевой переменной y
print("Проверка пройдена!")

## Разобьём данные

In [ ]:
from sklearn.model_selection import train_test_split

random_state = 42 # Фиксируем "случайность" чтобы получать одни и те же характеристики разбиения при разных запусках ноутбука

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=random_state, test_size=0.2)  
# stratify=y позволяет сохранить процентное соотношение между данными классов (сохраняем баланс классов)
# 20% данных - относим к проверочным данным

Прогоним тесты:

In [ ]:
assert X_train.shape == (11994, 19)
assert y_train.shape == (11994,)
assert X_test.shape == (2999, 19)
assert y_test.shape == (2999,)

assert np.sum(X_train) == 500668689
assert np.sum(X_test) == 125179430
print("Правильно!")

## Создадим первое решение

Создадим "метрику правильности" получаемых данных (по сути функция ошибки) 

In [ ]:
def metric(y_true, y_pred): 
    """Вычислим значение точности предсказания как характеристику 
    каппы Коэна (согласия/несогласия между двумя "оценщиками")"""
    from sklearn.metrics import cohen_kappa_score
    return cohen_kappa_score(y_true, y_pred, weights='quadratic')

In [ ]:
assert np.abs(1 - metric(y_train, y_train)) <= 1e-7
assert np.abs(1 - metric(y_test, y_test)) <= 1e-7
assert np.abs(metric(y_test, y_test + 1) - 0.7349020406) <= 1e-7
print("Есть контакт!")

Создаем первое поэтапное решение задачи (первый pipeline)

In [ ]:
def vanilla_pipeline(model):
    """Создаем программу, которая тренирует нашу модель на тренировочных данных и выдает значение точности предсказания"""
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return metric(y_test, y_pred)

Так как у нас много категориальных данных, то применить в чистом виде "k-nn Классификатор" (метод k-ближайших соседей (nearest neighbors)) будет не разумно. Тогда как применение модели деревьев решение - выглядит неплохим вариантом для базового решения.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

random_state = 49
rf = RandomForestClassifier(n_estimators=25, n_jobs=4, random_state=random_state)  # Количество деревьев=25, количество параллельных потоков=4
vanilla_pipeline(rf)

## Дальнейшее улучшение модели

In [ ]:
!ls ../input/petfinder-adoption-prediction/train_images/ | head -20  # Рассмотрим первые 20 фото
# Имя jpg файла представляет собой идентификатор животного - порядковый номер фото в объявлении.

Наибольшее влияние на выбор животного оказывает первое фото! Значит нам нужно оперировать первыми фотографиями (представить в виде вектора первые фото для дальнейших манипуляций)

In [ ]:
import os
image_list = sorted(os.listdir('../input/petfinder-adoption-prediction/train_images/'))  # Получаем image-list (список этих картинок)
image_list[:10] 

In [ ]:
from PIL import Image
image = Image.open('../input/petfinder-adoption-prediction/train_images/0008c5398-1.jpg')
image

Применим предобученную модель `torchvision.models` для перевода изображений в векторный вид:

In [ ]:
from torchvision import transforms

# Заполняем параметры библиотечной модели transforms
transform = transforms.Compose([            
 transforms.Resize(224),               
 transforms.ToTensor(),                     
 transforms.Normalize(                      
 mean=[0.485, 0.456, 0.406],            
 std=[0.229, 0.224, 0.225]              
 )])

In [ ]:
import torchvision.models as models

mobilenet = models.mobilenet_v2(pretrained=True).cuda() # По-умолчанию pytorch не делает вычисления на GPU, с помощью команды .cud

Так как веса модели скачиваются с внешнего источника, то в рамках правил данного соревнования мы не сможем сделать инференс данного ноутбука

Все готово, чтобы вычислить вложения. Для этого мы проделаем следующее:

* Трансформируем картинки в векторы
* Создаем пакеты содержащие данные об изображениях и преобразуем их с помощью .cuda()
* Сделаем предсказания
* Переведем предсказания в numpy-массив

In [ ]:
import torchvision.models as models
def calc_embedding(image):
    transformed = transform(image)
    batch = transformed.unsqueeze(0)
    predictions = mobilenet(batch.cuda())
    return predictions.cpu().detach().numpy().ravel()

In [ ]:
# Проверяем
embedding = calc_embedding(image)
embedding.std()

assert torch.cuda.current_device() == 0 # Проверяем что мы действительно используем .cuda
assert type(embedding) == np.ndarray # Проверяем что мы действительно конвертировали изображения в numpy.array
assert embedding.dtype == np.float32 # Проверяем, что формат эмбеддингов float32"
assert embedding.shape == (1000,) # Проверим, что эмбеддинги по форме предстяаляют собой кортеж (1000,)
print("Готово!")

Создаем функции для дальнейшей работы. Берем фотографию по id животного

In [ ]:
def _get_default_photo_path(pet_id):  
    '''Берем первую фотографию с "pet_id"'''
    return '../input/petfinder-adoption-prediction/train_images/%s-1.jpg' % pet_id

def does_pet_have_photo(pet_id): # Т. к. не у всех животных есть фото, то мы проверяем их (фото) наличие
    return os.path.exists(_get_default_photo_path(pet_id))

def photo_of_pet(pet_id):
    path = _get_default_photo_path(pet_id)
    return Image.open(path).convert('RGB')  # Т. к. есть ч/б и цветные изображения, то мы приводим их к одному "знаменателю" и ч/б

Получим вложения для тестового набора

In [ ]:
import tqdm

In [ ]:
train = pd.read_csv('../input/petfinder-adoption-prediction/train/train.csv')
train.PhotoAmt = train.PhotoAmt.astype(np.int64)

# Сохраним наши вложения здесь
embeddings = np.zeros((len(train), embedding.shape[0]), dtype=np.float32)

pet_ids = train.PetID

# Получим матрицу эмбеддингов (вложений картинок)
for i in tqdm.tqdm(range(len(train))):
    pet_id = pet_ids[i]
    
    if does_pet_have_photo(pet_id):
        embeddings[i] = calc_embedding(photo_of_pet(pet_id))

In [ ]:
assert embeddings.shape == (14993, 1000)
assert X.shape == (14993, 19)
print('Ничего не потеряно!')

Создаем новый датасет присоединив к существующему датафрейму наш, только что созданный embedding

In [ ]:
filter_text_columns(train)

# Заново создаем X и y массивы
X = np.array(train.iloc[:,:-1])
y = np.array(train.AdoptionSpeed)

X = np.hstack([X, embeddings])

In [ ]:
assert X.shape == (14993, 1019)
print('Валидно!')

Разобьём наш датасет на части train и test

In [ ]:
random_state = 49

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=random_state, test_size=0.2)

In [ ]:
rf = RandomForestClassifier(n_estimators=25, n_jobs=4, random_state=49)
vanilla_pipeline(rf)

Получим крайне низкие результаты, так как у нас 1000 картиночных фичей (в виде векторов) и 19 некартиночных. 
Возьмем только эмбеддинговые фичи и понизим у них размерность!

In [ ]:
X_train_feats = X_train[:,-1000:]
X_test_feats = X_test[:,-1000:]

Понизим размерности нашего датасета. Используем `TruncatedSVD`. Переведем X_train_feats и X_test_feats в новое шестимерное пространство.

## PCA. Применим метод главных компонент

In [ ]:
from sklearn.decomposition import TruncatedSVD

n_feats = 6  # Количество размерностей (6-чилсо подобранное эмперическим путем на основе анализа валидации)
random_state = 49

pca = TruncatedSVD(n_components=n_feats)
pca.fit(X_train_feats) # Обучаемся только на "Х", так как нам не важно y, мы только перегоняем наш исходный датасет в меньшую размерность
X_train_feats = pca.transform(X_train_feats) 
X_test_feats = pca.transform(X_test_feats)

In [ ]:
assert X_train.shape == (11994, 1019)
print('Правильно!')

Обучим нашу SVD на тренировочном датафрейме.
Изменим `X_train` и `X_test`, включив сжатые вложения.

In [ ]:
X_train = np.hstack([X_train[:,:19], X_train_feats])
X_test = np.hstack([X_test[:,:19], X_test_feats])

Проверим форму тренировочного array

In [ ]:
assert X_train.shape == (11994, 25)

In [ ]:
X_train

In [ ]:
rf = RandomForestClassifier(n_estimators=25, n_jobs=4, random_state=49)
vanilla_pipeline(rf)

Получили предсказательную точность на несколько процентов выше чем было

Увеличим результат применив CatBoost!

In [ ]:
from catboost import CatBoostClassifier

cb = CatBoostClassifier()
vanilla_pipeline(cb)

Сделаем предсказание на тестовых данных

In [ ]:
test = pd.read_csv('../input/petfinder-adoption-prediction/test/test.csv')
test.shape

In [ ]:
def _get_default_photo_path(pet_id):
    return '../input/petfinder-adoption-prediction/test_images/%s-1.jpg' % pet_id

def does_pet_have_photo(pet_id):
    return os.path.exists(_get_default_photo_path(pet_id))

def photo_of_pet(pet_id):
    path = _get_default_photo_path(pet_id)
    return Image.open(path).convert('RGB')

In [ ]:
# Сохраняем все вложения здесь
embeddings = np.zeros((len(test), 1000), dtype=np.float32)

pet_ids = test.PetID
for i in tqdm.tqdm(range(len(test))):
    pet_id = pet_ids[i]
    
    if does_pet_have_photo(pet_id):
        embeddings[i] = calc_embedding(photo_of_pet(pet_id))

In [ ]:
filter_text_columns(test)
test = test.astype(np.int64)

In [ ]:
assert test.shape == (3972, 19)
assert embeddings.shape == (3972, 1000)

In [ ]:
X_test_feats=pca.transform(embeddings) # Уменьшаем размерность

In [ ]:
X_test = test
X_test = np.hstack([X_test, X_test_feats]) # Присоеднияем 
X_test

In [ ]:
sample_submission = pd.read_csv('../input/petfinder-adoption-prediction/test/sample_submission.csv')
sample_submission.head()

In [ ]:
predictions = cb.predict(X_test)

In [ ]:
submission = sample_submission
submission['AdoptionSpeed'] = predictions
submission.to_csv('submission_v2.csv', index=False)